# **Teoremas de Dualidad y Funciones Convexas Lineales por Tramos**

In [124]:
%pip install -q amplpy

In [125]:
# Integración en Google Colab
from amplpy import AMPL, ampl_notebook

Primal = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


**Ejercicio 1.1:** Considera el problema de la dieta con el que hemos estado trabajando en la práctica anterior (en su variante más sencilla). Formula y resuelve su problema dual. *¿Qué situación práctica modela el problema dual? ¿Cómo lo interpretas?*

In [126]:
# Construimos el problema primal (que podemos coger de la práctica anterior):
Primal.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set ALIM;                       # Conjunto de alimentos
set NUTR;                       # Conjunto de nutrientes
param n_min {NUTR} >= 0;        # Cantidad mínima de nutrientes en la dieta
param coste {ALIM} >= 0;        # Coste de cada alimento (por cada 100 g)
param cont {ALIM, NUTR} >= 0;   # Contenido de nutriente en cada alimento

var x {ALIM} >= 0;              # Cantidad de cada alimento presente la dieta
                                # (múltiplo de 100 g)

minimize coste_total:
    sum{a in ALIM} coste[a]*x[a];
s.t. cant_minima {n in NUTR}:
    sum{a in ALIM} cont[a,n]*x[a] >= n_min[n];
""")

Instanciamos un nuevo objeto en AMPL que contendrá el modelo dual.

In [127]:
Dual = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


Y construimos el problema dual:

In [128]:
Dual.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set ALIM;                       # Conjunto de alimentos
set NUTR;                       # Conjunto de nutrientes
param n_min {NUTR} >= 0;        # Cantidad mínima de nutrientes en la dieta
param coste {ALIM} >= 0;        # Coste de cada alimento (por cada 100 g)
param cont {ALIM, NUTR} >= 0;   # Contenido de nutriente en cada alimento

var y {NUTR} >= 0;              # Variable dual asociada a la restricción que
                                # garantiza que se cumpla la cantidad mínima
                                # de nutrientes en la dieta

maximize beneficio_total:
    sum{n in NUTR} n_min[n]*y[n];
s.t. precio_max_nutrientes {a in ALIM}:
    sum{n in NUTR} cont[a,n]*y[n] <= coste[a];
""")

Introducimos los datos necesarios para resolver tanto el problema primal como su dual.

In [129]:
def preparar_datos():
    import pandas as pd
    import numpy as np

    # Creamos un data frame que contenga el coste de los alimentos
    alimentos_df = pd.DataFrame(
        [
            ("MaizA", 1),
            ("Avena", 0.5),
            ("MaizB", 2),
            ("Salvado", 1.2),
            ("Linaza", 3),
        ],
        columns=["ALIMENTO", "coste"],
    ).set_index("ALIMENTO")

    # Creamos un data frame para la cantidad mínima de nutrientes que se
    # requiere en la dieta
    nutr_df = pd.DataFrame(
        [
            ("DN", 74.2),
            ("DP", 14.7),
            ("Ca", 0.14),
            ("Ph", 0.55),
        ],
        columns=["NUTR", "n_min"],
    ).set_index("NUTR")

    # Creamos un data frame para la cantidad de nutrientes que cada
    # alimento contiene
    cont_df = pd.DataFrame(
        np.array(
            [
                [78.6, 70.1, 80.1, 67.2, 77.0],
                [6.50, 9.40, 8.80, 13.7, 30.4],
                [0.02, 0.09, 0.03, 0.14, 0.41],
                [0.27, 0.34, 0.30, 1.29, 0.86],
            ]
        ),
        columns= alimentos_df.index.to_list(),
        index  = nutr_df.index.to_list(),
    )
    cont_df = cont_df.transpose() # OJO, que los ejes (índices) deben coincidir
                                  # con los especificados en la definición del
                                  # parámetro "cont" en el modelo, (ALIM, NUTR)
    return alimentos_df, nutr_df, cont_df

In [130]:
# Cargamos los datos:

# Load the data from pandas.DataFrame objects:
alimentos_df, nutr_df, cont_df = preparar_datos()

# 1. Enviamos los datos de "alimentos_df" a AMPL e inicializamos el conjunto de índices "ALIM"
Primal.set_data(alimentos_df, "ALIM")
Dual.set_data(alimentos_df, "ALIM")
# 2. Enviamos los datos de "nutr_df" a AMPL e inicializamos el conjunto de índices "NUTR"
Primal.set_data(nutr_df, "NUTR")
Dual.set_data(nutr_df, "NUTR")
# 3. Fijamos los valores del parámetro "cont" usando "cont_df"
Primal.get_parameter("cont").set_values(cont_df)
Dual.get_parameter("cont").set_values(cont_df)

Resolvemos el problema primal:

In [131]:
Primal.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert Primal.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 0.7927691484
3 simplex iterations


In [132]:
x = Primal.get_variable("x") # Aquí el valor de "x" es convertido a un data frame de AMPL.
df_x = x.get_values().to_pandas() # Aquí el valor de "x" es convertido a un data frame de Pandas.
coste_total = Primal.get_objective("coste_total").value()
df_x = df_x.round(3)
print(df_x)
print(f"Coste total: {coste_total:.3f}")

         x.val
Avena    1.530
Linaza   0.000
MaizA    0.000
MaizB    0.000
Salvado  0.023
Coste total: 0.793


Y resolvemos el problema dual:

In [133]:
Dual.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert Dual.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 0.7927691484
4 simplex iterations


In [134]:
y = Dual.get_variable("y") # Aquí el valor de "y" es convertido a un data frame de AMPL.
df_y = y.get_values().to_pandas() # Aquí el valor de "y es convertido a un data frame de Pandas.
beneficio_total = Dual.get_objective("beneficio_total").value()
df_y = df_y.round(3)
print(df_y)
print(f"Beneficio total: {beneficio_total:.3f}")

    y.val
Ca  0.000
DN  0.000
DP  0.032
Ph  0.593
Beneficio total: 0.793


**Ejercicio 1.2**: Comprueba que los valores para las variables duales que has obtenido coinciden con las que puedes extraer directamente del "solver" utilizando amplpy.

In [135]:
Primal.display("cant_minima.dual")

cant_minima.dual [*] :=
Ca  0
DN  0
DP  0.0317354
Ph  0.593198
;



In [136]:
cant_minima_duales = Primal.get_constraint("cant_minima").get_values("dual").to_pandas()
cant_minima_duales = cant_minima_duales.round(3)
print(cant_minima_duales)







    cant_minima.dual
Ca             0.000
DN             0.000
DP             0.032
Ph             0.593


**Ejercicio 1.3**: Utiliza el problema de la dieta y su problema dual para comprobar que se satisfacen los resultados de dualidad vistos en clase. En concreto:


*   Determina diferentes pares de soluciones factibles para los problemas primal y dual y verifica que se tiene dualidad débil.
*   Determina diferentes pares de soluciones óptimas para ambos problemas (modificando alguno de sus datos de entrada) y comprueba que se verifica *dualidad fuerte* y *holgura complementaria*.
* Modifica los datos de entrada para hacer que alguno de los dos problemas sea ilimitado y comprueba que el otro resulta infactible en tal caso.




**Problema 2**: A continuación, vamos a modelar, de forma muy simplificada, el funcionamiento de un mercado eléctrico. Supón que este mercado lo componen $N_{G}$ productores de electricidad que han de satisfacer una demanda $D$. Para ello, cada productor o generador de electricidad incurre en un coste que viene dado por una *función convexa lineal por tramos*. Este tipo de función queda caracterizada por un número de tramos $N_{p_{i}}$ (que, en general, dependerá de cada productor $i = 1, \ldots, N_{G}$) y sus correspondientes longitudes y pendientes $(l_{i,j}, m_{i,j}), \forall i = 1, \ldots, N_{G}, \forall j = 1, \ldots, N_{p_{i}}$ (puesto que cada tramo es lineal). Asumiremos, además, que la función convexa lineal por tramos que describe el coste de producción de cada generador pasa por el 0 (esto es, que el productor no incurre en coste alguno si no produce electricidad).

1.   Formula un modelo de Programación Matemática para minimizar el coste total en el que incurre el mercado para satisfacer la demanda de electricidad $D$ (este modelo se conoce con el nombre de *modelo de cierre o liquidación de mercado*).


Instanciamos un nuevo objeto en AMPL que contendrá el modelo de liquidación del mercado.

In [137]:
Mercado = ampl_notebook(
    modules=["highs", "cplex"],  # Solvers que queremos instalar
    license_uuid="d1619e22-974c-4935-ad8a-2554d161c51d",  # licencia que usaremos (os la he mandado por correo)
)  # instanciamos el objeto "AMPL" con el que vamos a trabajar

Licensed to Bundle #6787.7245 expiring 20250228: 302-Optimization; 408-Operations Research, Prof. Juan Miguel Morales Gonz?lez, University of Malaga.


Construimos el modelo.

In [138]:
Mercado.eval(r"""
reset;                          # Para limpiar cualquier modelo previo
set Prod;                       # Conjunto de productores
set Tram;                       # Conjunto de tramos de las funciones de coste
param D >= 0;                   # Demanda del mercado
param lon {Prod, Tram} >= 0;    # Longitud de cada tramo
param slope {Prod, Tram} >= 0;  # Pendiente de cada tramo

var p {i in Prod, j in Tram} >= 0, <= lon[i,j];  # Potencia producida por cada
                                                 # productor en cada tramo

minimize coste_total:
    sum{i in Prod, j in Tram} slope[i,j]*p[i,j];
s.t. balance: sum{i in Prod, j in Tram} p[i,j] = D;
""")

2.   Resuelve el modelo anterior para liquidar un mercado con demanda $D = 200$ y compuesto por tres productores cuyas funciones de coste vienen caracterizadas en la tabla siguiente en la forma $(l_{i,j}, m_{i,j})$, $\forall i = 1, \ldots, N_{G}$, $\forall j = 1, \ldots, N_{p_{i}}$.

<center>

|  Productor    | Tramo 1   | Tramo 2  | Tramo 3 |
| ------------  | --------     | ---      | ---    |  
| 1            | (50, 10)      | (20, 15) | (30, 45)|
| 2            | (10, 5)       | (50, 26) | (60, 55)|   
| 3            | (100, 25)     | (50, 27) | (50, 30)|   

</center>

Introducimos los datos necesarios para resolver el modelo de cierre de mercado.

In [139]:
def preparar_datos_mercado():
    import pandas as pd
    import numpy as np

    # Conjunto de tramos o piezas:
    tramos = ["T1", "T2", "T3"]

    # Conjunto de productores:
    productores = ["G1", "G2", "G3"]

    # Demanda del mercado:
    demanda = 200

    # Creamos un data frame que contenga la longitud de los tramos
    lon_df = pd.DataFrame(
        np.array(
            [
                [50, 20, 30],
                [10, 50, 60],
                [100, 50, 50],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    # Creamos un data frame que contenga las pendientes de los tramos
    slope_df = pd.DataFrame(
        np.array(
            [
                [10, 15, 45],
                [5, 26, 55],
                [25, 27, 30],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    return demanda, tramos, productores, lon_df, slope_df


In [140]:
# Cargamos los datos:

# Load the data from pandas.DataFrame objects:
demanda, tramos, productores, lon_df, slope_df = preparar_datos_mercado()

Mercado.set["Prod"] = productores
Mercado.set["Tram"] = tramos
Mercado.get_parameter("lon").set_values(lon_df)
Mercado.get_parameter("slope").set_values(slope_df)
Mercado.get_parameter("D").set(demanda)


Resolvemos:

In [141]:
Mercado.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert Dual.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 3870
0 simplex iterations



3.   Calcula la variable dual asociada a la *ecuación de equilibrio (o de cierre de mercado)* para varios valores de la demanda eléctrica e interpreta su significado práctico.


In [142]:
p = Mercado.get_variable("p") # Producción por generador y tramo
df_p = p.get_values().to_pandas()
coste_total = Mercado.get_objective("coste_total").value()
precio = Mercado.get_constraint("balance").dual()
df_p = df_p.round(3)
print(df_p)
print(f"Coste total: {coste_total:.3f}")
print(f"Precio del mercado: {precio:.1f}")

# Resumimos lo que produce cada productor:
df_Pg = df_p.groupby(level='index0').sum()
print(df_Pg)

               p.val
index0 index1       
G1     T1         50
       T2         20
       T3          0
G2     T1         10
       T2         20
       T3          0
G3     T1        100
       T2          0
       T3          0
Coste total: 3870.000
Precio del mercado: 26.0
        p.val
index0       
G1         70
G2         30
G3        100


4.   Resuelve de nuevo el problema de cierre de mercado para las siguientes funciones de coste de producción:

<center>

|  Productor    | Tramo 1   | Tramo 2  | Tramo 3 |
| ------------  | --------     | ---      | ---    |  
| 1            | (50, 10)      | (20, 15) | (30, 45)|
| 2            | (10, 5)       | (50, 26) | (60, 15)|   
| 3            | (100, 25)     | (50, 5) | (50, 30)|   

</center>

Interpreta los resultados que has obtenido. ¿Notas algo raro en ellos? En caso afirmativo, ¿qué hay de raro y por qué?


In [143]:
def preparar_datos_mercado2():
    import pandas as pd
    import numpy as np

    # Conjunto de tramos o piezas:
    tramos = ["T1", "T2", "T3"]

    # Conjunto de productores:
    productores = ["G1", "G2", "G3"]

    # Demanda del mercado:
    demanda = 200

    # Creamos un data frame que contenga la longitud de los tramos
    lon_df = pd.DataFrame(
        np.array(
            [
                [50, 20, 30],
                [10, 50, 60],
                [100, 50, 50],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    # Creamos un data frame que contenga las pendientes de los tramos
    slope_df = pd.DataFrame(
        np.array(
            [
                [10, 15, 45],
                [5, 26, 15],
                [25, 5, 30],
            ]
        ),
        columns= tramos,
        index  = productores,
    )

    return demanda, tramos, productores, lon_df, slope_df

In [144]:
# Cargamos los datos:

# Load the data from pandas.DataFrame objects:
demanda, tramos, productores, lon_df, slope_df = preparar_datos_mercado2()

Mercado.set["Prod"] = productores
Mercado.set["Tram"] = tramos
Mercado.get_parameter("lon").set_values(lon_df)
Mercado.get_parameter("slope").set_values(slope_df)
Mercado.get_parameter("D").set(demanda)

In [145]:
Mercado.solve(solver="cplex") # Resolvemos con el solver "cplex"
assert Dual.solve_result == "solved"  # Comprobamos que el problema se ha resuelto correctamente

CPLEX 22.1.1:  - Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
 - CPXPARAM_Simplex_Display                         0
 - CPXPARAM_MIP_Display                             0
 - CPXPARAM_Barrier_Display                         0
CPLEX 22.1.1: optimal solution; objective 2250
3 simplex iterations


In [146]:
p = Mercado.get_variable("p") # Producción por generador y tramo
df_p = p.get_values().to_pandas()
coste_total = Mercado.get_objective("coste_total").value()
precio = Mercado.get_constraint("balance").dual()
df_p = df_p.round(3)
print(df_p)
print(f"Coste total: {coste_total:.3f}")
print(f"Precio del mercado: {precio:.1f}")

# Resumimos lo que produce cada productor:
df_Pg = df_p.groupby(level='index0').sum()
print(df_Pg)

               p.val
index0 index1       
G1     T1         50
       T2         20
       T3          0
G2     T1         10
       T2          0
       T3         60
G3     T1         10
       T2         50
       T3          0
Coste total: 2250.000
Precio del mercado: 25.0
        p.val
index0       
G1         70
G2         70
G3         60
